In [0]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install sk-video

In [0]:
# import data and load it in 

from glob import glob

how_does_that_sound_15 = glob('drive/My Drive/data-fr-005/how does that sound/15*.mp4')
# how_does_that_sound_1f = glob('drive/My Drive/data-fr-005/how does that sound/1f*.mp4')

how_does_that_sound = how_does_that_sound_15 

that_sounds_great_15 = glob('drive/My Drive/data-fr-005/that sounds great/15*.mp4')
# that_sounds_great_1f = glob('drive/My Drive/data-fr-005/that sounds great/1f*.mp4')

that_sounds_great = that_sounds_great_15

print(len(how_does_that_sound))
print(len(that_sounds_great))

In [0]:
x = []
y = []

import skvideo.io  
 
for i,vid in enumerate(that_sounds_great):
#   x.append(skvideo.io.vread(vid,outputdict={'-r': '15'}, num_frames=75))
  x.append(tf.to_float(skvideo.io.vread(vid,outputdict={'-r': '15'}, num_frames=75)))
  y.append([0, 3, 5, 6, 8, 8, 8, 1])
  if i % 10 == 0:
    print(i)
  
for i,vid in enumerate(how_does_that_sound):
  x.append(tf.to_float(skvideo.io.vread(vid,outputdict={'-r': '15'}, num_frames=75))) 
#   x.append(skvideo.io.vread(vid,outputdict={'-r': '15'}, num_frames=75))
  y.append([0, 7, 2, 3, 4, 8, 8, 1])
  if i % 10 == 0:
    print(i)

# print(np.asarray(x).shape)
# print(np.asarray(x2).shape)

In [0]:
len(x), len(y), x[0].shape
# x.nbytes

In [0]:
import os
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(x[1][1])
plt.colorbar()
plt.grid(False)

In [0]:
input_tensor = x
input_max_len = 75

target_tensor = np.asarray(y).astype(np.int32)
output_max_len = 8
output_vocab = 9

input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

In [0]:
# input_tensor_train[0]

In [0]:
# input_tensor_train = tf.to_float(input_tensor_train)
# target_tensor_train = target_tensor_train.astype(np.int32)

In [0]:
# BUFFER_SIZE = len(input_tensor_train)

BUFFER_SIZE = 64
BATCH_SIZE = 2
N_BATCH = BUFFER_SIZE//BATCH_SIZE

embedding_dim = 16
units = 16

vocab_tar_size = output_vocab

# dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
for i,j in dataset.take(2):
  print (j)

In [0]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    print('CuDNNGRU')
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
 
  else:
    print('GRU')
    return tf.layers.bidirectional(tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform'))

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(1000, embedding_dim)
        
        self.zero1 = tf.keras.layers.ZeroPadding3D(padding=(1, 2, 2), name='zero1')
        self.conv1 = tf.keras.layers.Conv3D(32, (3, 5, 5), strides=(1, 2, 2), kernel_initializer = tf.keras.initializers.he_normal(seed=None), name='conv1')
        self.batc1 = tf.keras.layers.BatchNormalization(name='batc1')
        self.actv1 = tf.keras.layers.Activation('relu', name='actv1')
        self.drop1 = tf.keras.layers.SpatialDropout3D(0.5)
        self.maxp1 = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')
 
        
        self.zero2 = tf.keras.layers.ZeroPadding3D(padding=(1, 2, 2), name='zero2')
        self.conv2 = tf.keras.layers.Conv3D(64, (3, 5, 5), strides=(1, 1, 1), kernel_initializer = tf.keras.initializers.he_normal(seed=None), name='conv2')
        self.batc2 = tf.keras.layers.BatchNormalization(name='batc2')
        self.actv2 = tf.keras.layers.Activation('relu', name='actv2')
        self.drop2 = tf.keras.layers.SpatialDropout3D(0.5)
        self.maxp2 = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')

        self.zero3 = tf.keras.layers.ZeroPadding3D(padding=(1, 1, 1), name='zero3')
        self.conv3 = tf.keras.layers.Conv3D(96, (3, 3, 3), strides=(1, 1, 1), kernel_initializer=tf.keras.initializers.he_normal(seed=None), name='conv3')
        self.batc3 = tf.keras.layers.BatchNormalization(name='batc3')
        self.actv3 = tf.keras.layers.Activation('relu', name='actv3')
        self.drop3 = tf.keras.layers.SpatialDropout3D(0.5)
        self.maxp3 = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')
        
        self.zero4 = tf.keras.layers.ZeroPadding3D(padding=(1, 1, 1), name='zero4')
        self.conv4 = tf.keras.layers.Conv3D(128, (3, 3, 3), strides=(1, 1, 1), kernel_initializer=tf.keras.initializers.he_normal(seed=None), name='conv3')
        self.batc4 = tf.keras.layers.BatchNormalization(name='batc4')
        self.actv4 = tf.keras.layers.Activation('relu', name='actv4')
        self.drop4 = tf.keras.layers.SpatialDropout3D(0.5)
        self.maxp4 = tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max4')
        
        self.resh1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        print(x.shape)
#         x = self.embedding(x)
       
        x = self.zero1(x)
#         print(x.shape)
        x = self.conv1(x)
#         print(x.shape)
        x = self.batc1(x)
        x = self.actv1(x)
        x = self.drop1(x)
        x = self.maxp1(x)
        
        print('conv1: '+str(x.shape))
        x = self.zero2(x)
        x = self.conv2(x)
        x = self.batc2(x)
        x = self.actv2(x)
        x = self.drop2(x)
        x = self.maxp2(x)
        
        print('conv2: '+str(x.shape))
        x = self.zero3(x)
        x = self.conv3(x)
        x = self.batc3(x)
        x = self.actv3(x)
        x = self.drop3(x)
        x = self.maxp3(x)
        
#         x = self.zero4(x)
#         x = self.conv4(x)
#         x = self.batc4(x)
#         x = self.actv4(x)
#         x = self.drop4(x)
#         x = self.maxp4(x)

#         print(str('in_resh: ')+str(x.shape))
#         print(x.shape)      
#         x = self.resh1(x)   -- causing error?
#         x = x.reshape(*x.shape[:1], -1)
        x = tf.reshape(x, [*x.shape[:1], -1])

        print(str('in_embedd: ')+str(x.shape))
        x = self.embedding(x)
    
#         print(str('in_gru: ')+str(x.shape))
        
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

# optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)



def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)


checkpoint_prefix = 'drive/My Drive/data-fr-005/ckpt1/'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
#         print(str('inp: ')+str(inp.shape))
        
        with tf.GradientTape() as tape:

            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
#             print('hidden: '+str(enc_hidden))
#             print('out: '+str(enc_output))

            dec_input = tf.expand_dims([0] * BATCH_SIZE, 1)   
            
#             print(dec_input)
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
               
#                 print('\ntarg[:, t: ' + str(targ[:, t]))
#                 print('predictions: ' + str(predictions))
                loss += loss_function(targ[:, t], predictions)
                
          
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
#         print(str('batch loss: ') + str(batch_loss))

        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
#         print(encoder.variables[0].shape)
#         print(decoder.variables[0].shape)
#         print(variables[0].shape)
#         print(len(decoder.variables))
#         print(len(encoder.variables))
      
        gradients = tape.gradient(loss, variables, output_gradients=None)
            
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 2 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for epoch {} sec\n'.format(time.time() - start))

In [0]:
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('objs.pkl', 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump([encoder, decoder], f)

# Getting back the objects:
# with open('objs.pkl') as f:  # Python 3: open(..., 'rb')
#     obj0, obj1, obj2 = pickle.load(f)

In [0]:
# input_tensor_val = tf.to_float(input_tensor_val)

In [0]:
def evaluate(sentence, encoder, decoder, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
#     sentence = preprocess_sentence(sentence)

#     inputs = [word2idx(vocab_en, i, 'en') for i in sentence.split(' ')]
#     inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
#     inputs = tf.convert_to_tensor(inputs)

    inputs = sentence
    
    result = ''

    hidden = [tf.zeros((1, units))]
#     hidden = encoder.initialize_hidden_state()

    enc_out, enc_hidden = encoder(inputs, hidden)
    
#     print('enc_out: '+str(enc_out))
#     print('enc_hidden: '+str(enc_hidden))

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([0], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
#         attention_weights = tf.reshape(attention_weights, (-1, ))
#         attention_plot[t] = attention_weights.numpy()
        
#         print('predictions: '+str(predictions[0]))

        predicted_id = tf.argmax(predictions[0]).numpy()
        
#         print(str('predicted ID: ')+ str(predicted_id))

        result += str(predicted_id) + ' '

        if predicted_id == 1:
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot
#     return result

In [0]:
for val in range(32):
  result, sentence, attention_plot = evaluate(tf.expand_dims(input_tensor_val[val],0), encoder, decoder, 75, 7)
  print(str('result is: ') + str(result) + str(target_tensor_val[val]))